<a href="https://colab.research.google.com/github/Dobenkss/MLlib/blob/main/MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Abhipraya Rasendriya Atma Ardany <br>
NIM : 24.21.1587 <br>

LINK : https://colab.research.google.com/drive/1M9duZ3ymcpmTckicRMP4MREb_VtpQNsi?usp=sharing

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [ ]:
spark = SparkSession.builder.appName("DiabetesClassification").getOrCreate()

In [ ]:
data = spark.read.csv("diabetes.csv", header=True, inferSchema=True)

data.show(5)
data.printSchema()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nul

In [ ]:
feature_columns = [col for col in data.columns if col != "Outcome"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

lr = LogisticRegression(labelCol="Outcome", featuresCol="features")

pipeline = Pipeline(stages=[assembler, lr])

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="Outcome", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"AUC: {auc}")

AUC: 0.8619186046511628


In [ ]:
predictions.select("Outcome", "prediction", "probability").show(10)

+-------+----------+--------------------+
|Outcome|prediction|         probability|
+-------+----------+--------------------+
|      0|       0.0|[0.95527793179914...|
|      0|       0.0|[0.93520104500295...|
|      0|       0.0|[0.87809674497398...|
|      0|       0.0|[0.98758412015116...|
|      0|       0.0|[0.96359923987192...|
|      0|       0.0|[0.95176906432069...|
|      0|       0.0|[0.99288479201560...|
|      0|       0.0|[0.82433460856301...|
|      0|       0.0|[0.87768387780641...|
|      1|       0.0|[0.86713492820924...|
+-------+----------+--------------------+
only showing top 10 rows

